### Load tensorflow

In [0]:
import tensorflow as tf
tf.reset_default_graph()
tf.set_random_seed(42)

### Read training and test data

In [0]:
import pandas as pd
train_df = pd.read_csv('data/flower_photos/train.csv')
test_df = pd.read_csv('data/flower_photos/test.csv')

In [3]:
#Get class names
class_names = train_df['class'].unique().tolist()
print('Flower classes: ', class_names)

Flower classes:  ['tulips', 'daisy', 'sunflowers', 'dandelion', 'roses']


In [0]:
#Define some parameters
img_size = 60
img_depth = 3  

### Build batch generator

In [0]:
import numpy as np

In [0]:
def batch_generator(df, batchsize=32, train_mode=True):   
    
    img_generator= tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                                   width_shift_range=0.2,
                                                                   height_shift_range=0.2,
                                                                   horizontal_flip=True)
    
    while True:       
        
        
        #Generate random numbers to pick images from dataset
        batch_nums = np.random.randint(0,df.shape[0],batchsize)
        
        #Initialize batch images array
        batch_images = np.zeros((batchsize,img_size, img_size,img_depth))
        
        #Initiate batch label array
        batch_labels = np.zeros((batchsize, len(class_names)))
        
        for i in range(batchsize):
            
            #Load image
            flower_image =  tf.keras.preprocessing.image.load_img('data/' + 
                                                                  df.loc[batch_nums[i]]['image_file'],
                                                                  target_size=(img_size, img_size))
            #Convert to array
            flower_image = tf.keras.preprocessing.image.img_to_array(flower_image)
            
            if(train_mode):
                #Apply transform
                flower_image =  img_generator.random_transform(flower_image)
            
            
            #Get the class
            img_class = df.loc[batch_nums[i]]['class']
            #Conver class to number
            img_class = class_names.index(img_class)
            #Convert class to one hot encoding
            img_class = tf.keras.utils.to_categorical(img_class, num_classes=len(class_names))
            
            #Update batch images and class arrays
            batch_images[i] = flower_image
            batch_labels[i] = img_class        
        
        yield batch_images, batch_labels          

### Build CNN Model

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape=(img_size,img_size,3,)))
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))

#Output layer
model.add(tf.keras.layers.Dense(len(class_names), activation='softmax'))

W0801 20:43:28.136089 139705549719424 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Train the model

In [0]:
#Create train and test generator
batchsize = 64
train_generator = batch_generator(train_df, batchsize=batchsize) #batchsize can be changed
test_generator = batch_generator(test_df, batchsize=batchsize, train_mode=False)

In [0]:
model.fit_generator(train_generator, 
                          epochs=50,
                          steps_per_epoch= train_df.shape[0]//batchsize,
                          validation_data=test_generator,
                          validation_steps = test_df.shape[0]//batchsize)

Epoch 1/50
45/45 [==============================] - 17s 376ms/step - loss: 1.7677 - acc: 0.4049 - val_loss: 1.5056 - val_acc: 0.4403
Epoch 2/50
45/45 [==============================] - 13s 295ms/step - loss: 1.2121 - acc: 0.4903 - val_loss: 1.2243 - val_acc: 0.4844
Epoch 3/50
45/45 [==============================] - 13s 286ms/step - loss: 1.1106 - acc: 0.5437 - val_loss: 0.9998 - val_acc: 0.5625
Epoch 4/50
45/45 [==============================] - 13s 293ms/step - loss: 1.0726 - acc: 0.5778 - val_loss: 1.0739 - val_acc: 0.5952
Epoch 5/50
45/45 [==============================] - 13s 288ms/step - loss: 1.0013 - acc: 0.6167 - val_loss: 0.8939 - val_acc: 0.6477
Epoch 6/50
45/45 [==============================] - 13s 288ms/step - loss: 0.9563 - acc: 0.6361 - val_loss: 0.8925 - val_acc: 0.6250
Epoch 7/50
45/45 [==============================] - 13s 289ms/step - loss: 0.9120 - acc: 0.6392 - val_loss: 0.9153 - val_acc: 0.6534
Epoch 8/50
45/45 [==============================] - 13s 283ms/step - 